In [ ]:
%mkdir -p /content/drive/MyDrive/GSE116198
%cd /content/drive/MyDrive/GSE116198

/content/drive/MyDrive/GSE116198


In [ ]:
%%bash

urls=(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE116198&format=file&file=GSE116198%5FGImap%5FsgRNA%5Fsequences%5Fbarcodes%2Etxt%2Egz"
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE116198&format=file&file=GSE116198%5FsgRNA%5Fpair%5Fphenotypes%2Etxt%2Egz"
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE116198&format=file&file=GSE116198%5FsgRNA%5Fpair%5Fread%5Fcounts%2Etxt%2Egz"
)

for url in "${urls[@]}"
do
   filename=$(echo "$url" | sed -n 's/.*file=\(.*\)/\1/p' | sed 's/%5F/_/g' | sed 's/%2E/./g')

   curl -o "$filename" "$url"

   echo "Downloaded: $filename"
done

Downloaded: GSE116198_GImap_sgRNA_sequences_barcodes.txt.gz
Downloaded: GSE116198_sgRNA_pair_phenotypes.txt.gz
Downloaded: GSE116198_sgRNA_pair_read_counts.txt.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36505  100 36505    0     0   118k      0 --:--:-- --:--:-- --:--:--  118k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  106M  100  106M    0     0  59.5M      0  0:00:01  0:00:01 --:--:-- 59.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.7M  100 45.7M    0     0  39.3M      0  0:00:01  0:00:01 --:--:-- 39.4M


In [ ]:
%ls

GSE116198_GImap_sgRNA_sequences_barcodes.txt.gz  GSE116198_sgRNA_pair_phenotypes.txt.gz
GSE116198.ipynb                                  GSE116198_sgRNA_pair_read_counts.txt.gz


In [ ]:
!zcat GSE116198_GImap_sgRNA_sequences_barcodes.txt.gz | head
!zcat GSE116198_sgRNA_pair_phenotypes.txt.gz | wc -l

291	AARS2_+_44281044.23-P1P2	AARS2_+_44281044.23-P1P2	Y		GGCTACGATGGCAGCGTCAG	AATATGTTCTCATAGG	TATTGGCATCATATGG	pLG18
292	AATF_-_35306286.23-P1P2	AATF_-_35306286.23-P1P2	Y		GAGTGGCCGGTCCAGAGCTG	CCGCCGCCGCCATTAG	ACCAACCTGACGCAGC	pLG18
294	ABCB7_+_74375984.23-P1P2	ABCB7_+_74375984.23-P1P2	Y		GTAGCGGCTCAGGTCCGCAG	GCGAAAGGCACTGTAT	TTTTTAATACCCTCAG	pLG18
295	ABCB7_-_74376019.23-P1P2	ABCB7_-_74376019.23-P1P2	Y		GAGCCGCTAACAGAGACTAA	GCCGCGATAGTCTGAA	GTACGCAGAGGTTTCC	pLG18
296	ACTL6A_+_179280849.23-P1P2	ACTL6A_+_179280849.23-P1P2	Y		GCGGCGCGACTCCTACCCTA	CATTTGATATTGTGTC	ATGCCCGTCGGAACAC	pLG18
298	ACTR10_+_58666938.23-P1P2	ACTR10_+_58666938.23-P1P2	Y		GTCCCCAGGCCCTCGTAGAG	CCTTTCGCATCGCATC	CCCGATCCTGATGCTA	pLG18
299	ACTR10_+_58666892.23-P1P2	ACTR10_+_58666892.23-P1P2	Y		GAAGAGGGTCGCAGTCTCCG	GCATGTCAGCTTTGGG	TACTGGACGTAGATGT	pLG18
300	ADAT2_+_143771782.23-P1P2	ADAT2_+_143771782.23-P1P2	Y		GTGGCTGGGTATGGAGGCGA	TTTAGGTTGTGTGCAC	TACAGGGTACACTGAC	pLG18
301	ADAT2_+_143771734.23-P1P2	ADAT2_+_143771734.

In [ ]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
counts = pd.read_csv('GSE116198_sgRNA_pair_read_counts.txt.gz', sep='\t', index_col=0, header=[0, 1, 2, 3, 4])
counts.head()

K562  \
                                                              barcode   
                                                                  cyc   
                                                                 rep1   
combined name                                      Unnamed: 1_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2              153.0   
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2              570.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2               324.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2               170.0   
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2              210.0   

                                                                       \
                                                                        
                                                                        
                                                                 rep2   
combined name                                      Unnamed: 2_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2              180.0   
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2              568.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2               180.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2               254.0   
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2              207.0   

                                                                       \
                                                                sgRNA   
                                                                  cyc   
                                                                 rep1   
combined name                                      Unnamed: 3_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2              248.0   
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2              748.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2               330.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2               183.0   
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2              215.0   

                                                                       \
                                                                        
                                                                        
                                                                 rep2   
combined name                                      Unnamed: 4_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2              285.0   
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2              513.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2               218.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2               306.0   
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2              208.0   

                                                                       \
                                                            tripleseq   
                                                                  cyc   
                                                                 rep1   
combined name                                      Unnamed: 5_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2              113.0   
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2              422.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2               213.0   
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2               130.0   
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2              100.0   

                                                                       \
                                                                        
                                                                        
                                                                 rep2   
combined name                                      Unnamed: 6_level_4   
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23

In [ ]:
counts.columns = counts.columns.map(lambda x: '_'.join(x[:-1]))

In [ ]:
counts.head()

,K562_barcode_cyc_rep1,K562_barcode_cyc_rep2,K562_sgRNA_cyc_rep1,K562_sgRNA_cyc_rep2,K562_tripleseq_cyc_rep1,K562_tripleseq_cyc_rep2,K562_barcode_T0_rep1,K562_barcode_T0_rep2,Jurkat_barcode_T0_rep1,Jurkat_barcode_T0_rep2,Jurkat_barcode_cyc_rep1,Jurkat_barcode_cyc_rep2,Jurkat_sgRNA_T0_rep1,Jurkat_sgRNA_T0_rep2,Jurkat_sgRNA_cyc_rep1,Jurkat_sgRNA_cyc_rep2,Jurkat_tripleseq_T0_rep1,Jurkat_tripleseq_T0_rep2,Jurkat_tripleseq_cyc_rep1,Jurkat_tripleseq_cyc_rep2
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2,153.0,180.0,248.0,285.0,113.0,99.0,288,442,455.0,438.0,510.0,364.0,827.0,705.0,637.0,576.0,197.0,165.0,221.0,160.0
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2,570.0,568.0,748.0,513.0,422.0,344.0,641,712,1124.0,868.0,872.0,843.0,1051.0,825.0,805.0,824.0,598.0,438.0,465.0,486.0
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2,324.0,180.0,330.0,218.0,213.0,117.0,354,363,482.0,386.0,377.0,377.0,545.0,424.0,391.0,382.0,229.0,167.0,184.0,202.0
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2,170.0,254.0,183.0,306.0,130.0,171.0,264,427,525.0,448.0,384.0,340.0,713.0,503.0,450.0,336.0,302.0,205.0,210.0,160.0
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2,210.0,207.0,215.0,208.0,100.0,110.0,559,458,668.0,563.0,171.0,206.0,547.0,416.0,119.0,135.0,255.0,216.0,48.0,81.0


In [ ]:
phenotypes = pd.read_csv('GSE116198_sgRNA_pair_phenotypes.txt.gz', sep='\t', index_col=0, header=[0, 1, 2, 3,])
phenotypes.columns = phenotypes.columns.map(lambda x: '_'.join(x[:-1]))
phenotypes.head()

,Jurkat_Barcode Sequencing_Replicate 1,Jurkat_Barcode Sequencing_Replicate 2,Jurkat_Barcode Sequencing_Replicate average,Jurkat_Triple Sequencing_Replicate 1,Jurkat_Triple Sequencing_Replicate 2,Jurkat_Triple Sequencing_Replicate average,Jurkat_sgRNA Sequencing_Replicate 1,Jurkat_sgRNA Sequencing_Replicate 2,Jurkat_sgRNA Sequencing_Replicate average,K562_Barcode Sequencing_Replicate 1,K562_Barcode Sequencing_Replicate 2,K562_Barcode Sequencing_Replicate average,K562_Triple Sequencing_Replicate 1,K562_Triple Sequencing_Replicate 2,K562_Triple Sequencing_Replicate average,K562_sgRNA Sequencing_Replicate 1,K562_sgRNA Sequencing_Replicate 2,K562_sgRNA Sequencing_Replicate average
AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2,-0.068827,-0.191852,-0.130340,-0.083255,-0.208204,-0.145730,-0.173328,-0.221788,-0.197558,-0.595598,-0.549577,-0.572587,-0.585627,-0.591593,-0.588610,-0.559608,-0.516329,-0.537969
AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2,-0.153986,-0.157884,-0.155935,-0.166878,-0.178913,-0.172896,-0.174806,-0.177491,-0.176148,-0.493740,-0.427449,-0.460595,-0.486491,-0.457066,-0.471779,-0.497744,-0.496563,-0.497153
AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2,-0.151345,-0.156563,-0.153954,-0.157906,-0.161295,-0.159601,-0.189165,-0.200021,-0.194593,-0.487587,-0.513158,-0.500373,-0.503173,-0.526199,-0.514686,-0.543756,-0.528751,-0.536254
AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2,-0.166791,-0.211653,-0.189222,-0.190921,-0.254313,-0.222617,-0.218992,-0.265438,-0.242215,-0.557354,-0.480822,-0.519088,-0.541068,-0.489050,-0.515059,-0.602681,-0.496894,-0.549788
AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2,-0.404775,-0.369940,-0.387358,NaN,NaN,NaN,-0.455999,-0.418618,-0.437308,-0.668027,-0.530982,-0.599504,-0.743988,-0.579960,-0.661974,-0.723221,-0.580268,-0.651744


In [ ]:
guide_infos = pd.read_csv('GSE116198_GImap_sgRNA_sequences_barcodes.txt.gz', sep='\t', index_col=0, header=None)
gene_map = dict(zip(guide_infos[1], guide_infos[8]))
guide_infos.head()

,1,2,3,4,5,6,7,8
0,,,,,,,,
291,AARS2_+_44281044.23-P1P2,AARS2_+_44281044.23-P1P2,Y,NaN,GGCTACGATGGCAGCGTCAG,AATATGTTCTCATAGG,TATTGGCATCATATGG,pLG18
292,AATF_-_35306286.23-P1P2,AATF_-_35306286.23-P1P2,Y,NaN,GAGTGGCCGGTCCAGAGCTG,CCGCCGCCGCCATTAG,ACCAACCTGACGCAGC,pLG18
294,ABCB7_+_74375984.23-P1P2,ABCB7_+_74375984.23-P1P2,Y,NaN,GTAGCGGCTCAGGTCCGCAG,GCGAAAGGCACTGTAT,TTTTTAATACCCTCAG,pLG18
295,ABCB7_-_74376019.23-P1P2,ABCB7_-_74376019.23-P1P2,Y,NaN,GAGCCGCTAACAGAGACTAA,GCCGCGATAGTCTGAA,GTACGCAGAGGTTTCC,pLG18
296,ACTL6A_+_179280849.23-P1P2,ACTL6A_+_179280849.23-P1P2,Y,NaN,GCGGCGCGACTCCTACCCTA,CATTTGATATTGTGTC,ATGCCCGTCGGAACAC,pLG18


In [ ]:
guide_infos[8].unique()

array(['pLG18'], dtype=object)

In [ ]:
def assign_gene(guides):
    return (
        "+".join([
            guide.split('_')[0] for guide in guides.split('++')
        ])
    )

In [ ]:
phenotypes.shape

(922455, 18)

In [ ]:
adata = ad.AnnData(counts.T, uns={'phenotypes': phenotypes.T})
adata.var['gene_name'] = adata.var_names.to_series().apply(assign_gene)
adata.var['guides_name'] = adata.var_names
adata.var.index = adata.var['gene_name']

adata.obs['condition'] = adata.obs_names.to_series().str.split('_').str[0]
adata.obs['condition'].value_counts()

,count
condition,
Jurkat,12
K562,8


In [ ]:
import numpy as np

adata.obs['perturbation_name'] = counts.T.apply(lambda x: "+".join(np.unique(adata.var_names.values[(x > 0).values])), axis=1)

In [ ]:
np.unique(adata.var_names.values)

array(['AARS2+AARS2', 'AARS2+AATF', 'AARS2+ABCB7', ..., 'negative+ZNRD1',
       'negative+ZWINT', 'negative+negative'], dtype=object)

In [ ]:
adata.obs.head()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs['cancer_type'] = adata.obs['condition'].apply(
    lambda x: 'non-cancer' if x == 'Jurkat' else 'Leukemia'
)
adata.obs['cell_type'] = adata.obs['condition'].apply(
    lambda x: 'T cell' if x == 'Jurkat' else 'leukemia cell'
)
adata.obs['organism'] = ORGANISMS[0]

In [ ]:
adata.var.drop(columns=['gene_name'], inplace=True)

In [18]:
adata.uns.pop('phenotypes')

,AARS2_+_44281027.23-P1P2++AARS2_+_44281027.23-P1P2,AARS2_+_44281027.23-P1P2++AARS2_+_44281044.23-P1P2,AARS2_+_44281027.23-P1P2++AATF_-_35306286.23-P1P2,AARS2_+_44281027.23-P1P2++AATF_-_35306346.23-P1P2,AARS2_+_44281027.23-P1P2++ABCB7_+_74375984.23-P1P2,AARS2_+_44281027.23-P1P2++ABCB7_-_74376019.23-P1P2,AARS2_+_44281027.23-P1P2++ACTL6A_+_179280849.23-P1P2,AARS2_+_44281027.23-P1P2++ACTR10_+_58666892.23-P1P2,AARS2_+_44281027.23-P1P2++ACTR10_+_58666933.23-P1P2,AARS2_+_44281027.23-P1P2++ACTR10_+_58666938.23-P1P2,...,negative_control-10927++negative_control-10057,negative_control-10927++negative_control-10081,negative_control-10927++negative_control-10094,negative_control-10927++negative_control-10095,negative_control-10927++negative_control-10100,negative_control-10927++negative_control-10114,negative_control-10927++negative_control-10115,negative_control-10927++negative_control-10130,negative_control-10927++negative_control-10899,negative_control-10927++negative_control-10927
Jurkat_Barcode Sequencing_Replicate 1,-0.068827,-0.153986,-0.151345,-0.166791,-0.404775,-0.429693,-0.274204,-0.183039,-0.187615,-0.240228,...,0.090897,-0.006745,0.071340,0.115180,0.002753,0.060669,0.024915,0.034169,0.084136,0.042267
Jurkat_Barcode Sequencing_Replicate 2,-0.191852,-0.157884,-0.156563,-0.211653,-0.369940,-0.400550,-0.293079,-0.107039,-0.178641,-0.244196,...,-0.007342,-0.000630,0.059036,0.046776,0.060136,0.002052,-0.012947,-0.030556,0.027901,-0.121504
Jurkat_Barcode Sequencing_Replicate average,-0.130340,-0.155935,-0.153954,-0.189222,-0.387358,-0.415121,-0.283642,-0.145039,-0.183128,-0.242212,...,0.041778,-0.003687,0.065188,0.080978,0.031445,0.031361,0.005984,0.001806,0.056019,-0.039618
Jurkat_Triple Sequencing_Replicate 1,-0.083255,-0.166878,-0.157906,-0.190921,NaN,-0.506353,-0.274444,-0.218929,-0.207858,-0.259250,...,0.111505,0.004246,0.077053,0.093219,0.003654,0.071060,0.024301,0.017719,0.089119,0.000158
Jurkat_Triple Sequencing_Replicate 2,-0.208204,-0.178913,-0.161295,-0.254313,NaN,-0.376683,-0.312957,-0.161728,-0.191755,-0.274409,...,-0.048263,-0.025826,0.102723,0.073292,0.063988,0.020191,0.015137,-0.005082,0.054092,-0.130922
Jurkat_Triple Sequencing_Replicate average,-0.145730,-0.172896,-0.159601,-0.222617,NaN,-0.441518,-0.293701,-0.190328,-0.199807,-0.266830,...,0.031621,-0.010790,0.089888,0.083255,0.033821,0.045626,0.019719,0.006318,0.071605,-0.065382
Jurkat_sgRNA Sequencing_Replicate 1,-0.173328,-0.174806,-0.189165,-0.218992,-0.455999,-0.466785,-0.291957,-0.192752,-0.199151,-0.208035,...,0.106413,0.022958,0.044412,0.080612,-0.001580,0.054041,0.019953,0.026612,0.057540,-0.004123
Jurkat_sgRNA Sequencing_Replicate 2,-0.221788,-0.177491,-0.200021,-0.265438,-0.418618,-0.390190,-0.312702,-0.146440,-0.224667,-0.227850,...,0.015185,-0.020301,0.058714,0.068031,0.036357,0.015855,0.015574,0.022809,0.026576,-0.029465
Jurkat_sgRNA Sequencing_Replicate average,-0.197558,-0.176148,-0.194593,-0.242215,-0.437308,-0.428488,-0.302329,-0.169596,-0.211909,-0.217942,...,0.060799,0.001328,0.051563,0.074322,0.017389,0.034948,0.017763,0.024710,0.042058,-0.016794
K562_Barcode Sequencing_Replicate 1,-0.595598,-0.493740,-0.487587,-0.557354,-0.668027,-0.632134,-0.527243,-0.714728,-0.668083,-0.598095,...,0.094186,-0.027100,0.112266,0.108738,-0.019479,0.020515,0.037911,-0.052832,-0.038434,-0.218335


In [20]:
adata.uns['phenotypes'] = phenotypes
adata.write_h5ad('GSE116198_processed.h5ad')